# 1. Data Processing: 

In [45]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, Embedding, Reshape, Concatenate, Dropout
import os, glob
import numpy as np
from keras.models import model_from_yaml
from keras.preprocessing import image
from keras.preprocessing.image import array_to_img, img_to_array, load_img

In [46]:
#a)
train_datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True)

In [47]:
#b)
train_generator = train_datagen.flow_from_directory(
        'Downloads/dataset_train', 
        target_size = (64, 64),  
        batch_size = 32,
        class_mode = 'categorical') 


Found 88 images belonging to 4 classes.


c) Take a look at your training set: 
1. What is the image shape of each training observation?

Answer: The original image shape is (800,800,3) (see code below). The traget image size is (64,64,3) as we specify the target size in the train generator above. 

2. How many total classes do we need to predict on? 

Answer: There are total of 4 classes.

In [48]:

img = load_img('Downloads/dataset_train/category 1/1010.png')
x = img_to_array(img)
x.shape

(800, 800, 3)

# 2. Initial Classifier Build

In [49]:
classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape=(64,64,3), activation = "relu"))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(64, (3, 3), activation= 'relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())
classifier.add(Dense(units = 128, activation='relu'))
classifier.add(Dense(units = 4, activation='softmax'))
classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 3. Model Runs

## 3-1. steps_per_epoch: 10, epochs: 10

In [50]:
#a)
classifier.fit_generator(train_generator,steps_per_epoch=10,epochs=10)

Epoch 1/10
10/10 [==============================] - 5s 454ms/step - loss: 1.0350 - accuracy: 0.6858
Epoch 2/10
10/10 [==============================] - 4s 398ms/step - loss: 0.2008 - accuracy: 0.9444
Epoch 3/10
10/10 [==============================] - 4s 413ms/step - loss: 0.1153 - accuracy: 0.9662
Epoch 4/10
10/10 [==============================] - 5s 466ms/step - loss: 0.0612 - accuracy: 0.9797
Epoch 5/10
10/10 [==============================] - 4s 422ms/step - loss: 0.0358 - accuracy: 0.9861
Epoch 6/10
10/10 [==============================] - 4s 450ms/step - loss: 0.0202 - accuracy: 0.9932
Epoch 7/10
10/10 [==============================] - 4s 431ms/step - loss: 0.0217 - accuracy: 0.9896
Epoch 8/10
10/10 [==============================] - 4s 448ms/step - loss: 0.0098 - accuracy: 0.9966
Epoch 9/10
10/10 [==============================] - 4s 402ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 10/10
10/10 [==============================] - 4s 401ms/step - loss: 0.0064 - accuracy: 1.0000

In [51]:
#b)
# write model and model weights to disk
model_yaml = classifier.to_yaml()
with open("model_1.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_1.h5")
print("Saved model to disk")

Saved model to disk


In [52]:
#c)
import os, glob
import numpy as np
from keras.models import model_from_yaml
from keras.preprocessing import image
import tensorflow as tf

# load model from disk
yaml_file = open('Desktop/model_1.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
model = tf.keras.models.model_from_yaml(loaded_model_yaml)

# load weights into new model
model.load_weights("model_1.h5")
print("Loaded model from disk")

# test data path
img_dir = "Downloads/dataset_test" # Enter Directory of all images

# iterate over each test image
# make a prediction and add to results 
data_path = os.path.join(img_dir, '*g')
files = glob.glob(data_path)
data = []
results = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result = model.predict(img)
    r = np.argmax(result, axis=1)
    results.append(r)

results


Loaded model from disk


[array([3]),
 array([0]),
 array([1]),
 array([0]),
 array([1]),
 array([2]),
 array([1]),
 array([1])]

In [53]:
files

['Downloads/dataset_test/C033.png',
 'Downloads/dataset_test/1022.png',
 'Downloads/dataset_test/4011.png',
 'Downloads/dataset_test/1053.png',
 'Downloads/dataset_test/6051.png',
 'Downloads/dataset_test/4053.png',
 'Downloads/dataset_test/C014.png',
 'Downloads/dataset_test/6023.png']

In [39]:
#d)
print("train set class indices:",train_generator.class_indices)

correct_class = np.array([3,0,1,0,2,1,3,2])
import itertools
result1 = list(itertools.chain.from_iterable(results))
accuracy_rate1 = sum(correct_class == result1)/len(correct_class)
print("classification result", result1)
print("accuracy rate for model 1:", accuracy_rate1)

train set class indices: {'category 1': 0, 'category 2': 1, 'category 3': 2, 'category 4': 3}
classification result [3, 0, 1, 0, 1, 2, 1, 1]
accuracy rate for model 1: 0.5


## 3-2. steps_per_epoch: 10, epochs: 20

In [17]:
model_2 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=10,
            epochs=20)

Epoch 1/20
10/10 [==============================] - 4s 418ms/step - loss: 0.0068 - accuracy: 0.9965
Epoch 2/20
10/10 [==============================] - 4s 400ms/step - loss: 0.0159 - accuracy: 0.9899
Epoch 3/20
10/10 [==============================] - 5s 504ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 5s 516ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 4s 395ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 5s 455ms/step - loss: 8.8414e-04 - accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 4s 403ms/step - loss: 0.0066 - accuracy: 0.9966
Epoch 8/20
10/10 [==============================] - 4s 401ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 9/20
10/10 [==============================] - 4s 408ms/step - loss: 0.0132 - accuracy: 0.9932
Epoch 10/20
10/10 [==============================] - 4s 407ms/step - loss: 0.0071 - accuracy: 0.

In [18]:
#save model and weights
with open("model_2.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_2.h5")

In [19]:
#load model and weights
yaml_file2 = open('model_2.yaml', 'r')
loaded_model_yaml2 = yaml_file2.read()
yaml_file2.close()
model2 = tf.keras.models.model_from_yaml(loaded_model_yaml2)

model2.load_weights("model_2.h5")

In [40]:
# iterate over each test image
# make a prediction and add to results 
data = []
results2 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result2 = model2.predict(img)
    r2 = np.argmax(result2, axis=1)
    results2.append(r2)


result2 = list(itertools.chain.from_iterable(results2))
accuracy_rate2 = sum(correct_class == result2)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result2)
print("accuracy rate for model 2:", accuracy_rate2)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 1, 0, 1, 2, 1, 1]
accuracy rate for model 2: 0.5


# 3-3. steps_per_epoch: 10, epochs: 30

In [25]:
model_3 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=10,
            epochs=30)

Epoch 1/30
10/10 [==============================] - 5s 539ms/step - loss: 1.9436e-04 - accuracy: 1.0000
Epoch 2/30
10/10 [==============================] - 4s 431ms/step - loss: 9.7374e-05 - accuracy: 1.0000
Epoch 3/30
10/10 [==============================] - 4s 429ms/step - loss: 1.6439e-04 - accuracy: 1.0000
Epoch 4/30
10/10 [==============================] - 4s 420ms/step - loss: 2.5241e-04 - accuracy: 1.0000
Epoch 5/30
10/10 [==============================] - 4s 421ms/step - loss: 8.5637e-05 - accuracy: 1.0000
Epoch 6/30
10/10 [==============================] - 4s 434ms/step - loss: 1.1694e-04 - accuracy: 1.0000
Epoch 7/30
10/10 [==============================] - 5s 471ms/step - loss: 9.1273e-05 - accuracy: 1.0000
Epoch 8/30
10/10 [==============================] - 4s 397ms/step - loss: 9.0882e-05 - accuracy: 1.0000
Epoch 9/30
10/10 [==============================] - 4s 442ms/step - loss: 4.2279e-05 - accuracy: 1.0000
Epoch 10/30
10/10 [==============================] - 4s 422ms/st

In [26]:
#save model and weights
with open("model_3.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_3.h5")

In [27]:
#load model and weights
yaml_file3 = open('model_3.yaml', 'r')
loaded_model_yaml3 = yaml_file3.read()
yaml_file3.close()
model3 = tf.keras.models.model_from_yaml(loaded_model_yaml3)

model3.load_weights("model_3.h5")

In [41]:
# iterate over each test image
# make a prediction and add to results 
data = []
results3 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result3 = model3.predict(img)
    r3 = np.argmax(result3, axis=1)
    results3.append(r3)


result3 = list(itertools.chain.from_iterable(results3))
accuracy_rate3 = sum(correct_class == result3)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result3)
print("accuracy rate for model 3:", accuracy_rate3)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 1, 0, 1, 2, 1, 1]
accuracy rate for model 3: 0.5


# 3-4. steps_per_epoch: 30, epochs: 10

In [29]:
model_4 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=30,
            epochs=10)

Epoch 1/10
30/30 [==============================] - 13s 444ms/step - loss: 5.2155e-05 - accuracy: 1.0000
Epoch 2/10
30/30 [==============================] - 13s 435ms/step - loss: 3.1566e-05 - accuracy: 1.0000
Epoch 3/10
30/30 [==============================] - 14s 469ms/step - loss: 3.4368e-05 - accuracy: 1.0000
Epoch 4/10
30/30 [==============================] - 14s 457ms/step - loss: 5.5155e-05 - accuracy: 1.0000
Epoch 5/10
30/30 [==============================] - 13s 439ms/step - loss: 3.1630e-05 - accuracy: 1.0000
Epoch 6/10
30/30 [==============================] - 13s 444ms/step - loss: 1.8791e-05 - accuracy: 1.0000
Epoch 7/10
30/30 [==============================] - 13s 437ms/step - loss: 3.3689e-05 - accuracy: 1.0000
Epoch 8/10
30/30 [==============================] - 15s 512ms/step - loss: 2.5881e-05 - accuracy: 1.0000
Epoch 9/10
30/30 [==============================] - 14s 473ms/step - loss: 1.8813e-05 - accuracy: 1.0000
Epoch 10/10
30/30 [==============================] - 13

In [30]:
#save model and weights
with open("model_4.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_4.h5")

In [31]:
#load model and weights
yaml_file4 = open('model_4.yaml', 'r')
loaded_model_yaml4 = yaml_file4.read()
yaml_file4.close()
model4 = tf.keras.models.model_from_yaml(loaded_model_yaml4)

model4.load_weights("model_4.h5")

In [42]:
# iterate over each test image
# make a prediction and add to results 
data = []
results4 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result4 = model4.predict(img)
    r4 = np.argmax(result4, axis=1)
    results4.append(r4)


result4 = list(itertools.chain.from_iterable(results4))
accuracy_rate4 = sum(correct_class == result4)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result4)
print("accuracy rate for model 4:", accuracy_rate4)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 1, 0, 1, 2, 1, 1]
accuracy rate for model 4: 0.5


## 3-5. steps_per_epoch: 30, epochs: 20

In [33]:
model_5 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=30,
            epochs=20)

Epoch 1/20
30/30 [==============================] - 13s 426ms/step - loss: 2.6872e-05 - accuracy: 1.0000
Epoch 2/20
30/30 [==============================] - 12s 411ms/step - loss: 1.9473e-05 - accuracy: 1.0000
Epoch 3/20
30/30 [==============================] - 12s 404ms/step - loss: 2.4010e-05 - accuracy: 1.0000
Epoch 4/20
30/30 [==============================] - 12s 407ms/step - loss: 1.9326e-05 - accuracy: 1.0000
Epoch 5/20
30/30 [==============================] - 12s 408ms/step - loss: 1.6813e-05 - accuracy: 1.0000
Epoch 6/20
30/30 [==============================] - 12s 409ms/step - loss: 1.4616e-05 - accuracy: 1.0000
Epoch 7/20
30/30 [==============================] - 12s 408ms/step - loss: 9.8720e-06 - accuracy: 1.0000
Epoch 8/20
30/30 [==============================] - 12s 410ms/step - loss: 1.0139e-05 - accuracy: 1.0000
Epoch 9/20
30/30 [==============================] - 12s 406ms/step - loss: 1.4866e-05 - accuracy: 1.0000
Epoch 10/20
30/30 [==============================] - 12

In [34]:
#save model and weights
with open("model_5.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_5.h5")

In [35]:
#load model and weights
yaml_file5 = open('model_5.yaml', 'r')
loaded_model_yaml5 = yaml_file5.read()
yaml_file5.close()
model5 = tf.keras.models.model_from_yaml(loaded_model_yaml5)

model5.load_weights("model_5.h5")

In [44]:
# iterate over each test image
# make a prediction and add to results 
data = []
results5 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result5 = model5.predict(img)
    r5 = np.argmax(result5, axis=1)
    results5.append(r5)


result5 = list(itertools.chain.from_iterable(results5))
accuracy_rate5 = sum(correct_class == result5)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result5)
print("accuracy rate for model 5:", accuracy_rate5)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 1, 0, 1, 2, 1, 1]
accuracy rate for model 5: 0.5


## 3-6. steps_per_epoch: 30, epochs: 30

In [54]:
model_6 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=30,
            epochs=30)

Epoch 1/30
30/30 [==============================] - 13s 428ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 2/30
30/30 [==============================] - 12s 408ms/step - loss: 8.3944e-04 - accuracy: 1.0000
Epoch 3/30
30/30 [==============================] - 12s 413ms/step - loss: 0.0015 - accuracy: 1.0000
Epoch 4/30
30/30 [==============================] - 12s 411ms/step - loss: 3.9628e-04 - accuracy: 1.0000
Epoch 5/30
30/30 [==============================] - 12s 402ms/step - loss: 1.9551e-04 - accuracy: 1.0000
Epoch 6/30
30/30 [==============================] - 12s 405ms/step - loss: 3.7084e-04 - accuracy: 1.0000
Epoch 7/30
30/30 [==============================] - 13s 445ms/step - loss: 8.4064e-04 - accuracy: 1.0000
Epoch 8/30
30/30 [==============================] - 12s 406ms/step - loss: 3.5090e-04 - accuracy: 1.0000
Epoch 9/30
30/30 [==============================] - 12s 408ms/step - loss: 8.4436e-05 - accuracy: 1.0000
Epoch 10/30
30/30 [==============================] - 12s 409ms/

In [55]:
#save model and weights
with open("model_6.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_6.h5")

In [56]:
#load model and weights
yaml_file6 = open('model_6.yaml', 'r')
loaded_model_yaml6 = yaml_file6.read()
yaml_file6.close()
model6 = tf.keras.models.model_from_yaml(loaded_model_yaml6)

model6.load_weights("model_6.h5")

In [57]:
# iterate over each test image
# make a prediction and add to results 
data = []
results6 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result6 = model6.predict(img)
    r6 = np.argmax(result6, axis=1)
    results6.append(r6)


result6 = list(itertools.chain.from_iterable(results6))
accuracy_rate6 = sum(correct_class == result6)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result6)
print("accuracy rate for model 6:", accuracy_rate6)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 2, 0, 1, 2, 1, 1]
accuracy rate for model 6: 0.375


## 3-7. steps_per_epoch: 50, epochs: 10

In [58]:
model_7 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=50,
            epochs=10)

Epoch 1/10
50/50 [==============================] - 22s 450ms/step - loss: 7.8863e-06 - accuracy: 1.0000
Epoch 2/10
50/50 [==============================] - 21s 426ms/step - loss: 2.2137e-05 - accuracy: 1.0000
Epoch 3/10
50/50 [==============================] - 21s 426ms/step - loss: 1.0147e-05 - accuracy: 1.0000
Epoch 4/10
50/50 [==============================] - 23s 455ms/step - loss: 7.1070e-06 - accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - 23s 459ms/step - loss: 5.4745e-06 - accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 22s 432ms/step - loss: 9.8681e-06 - accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 21s 429ms/step - loss: 4.8397e-06 - accuracy: 1.0000
Epoch 8/10
50/50 [==============================] - 25s 493ms/step - loss: 7.0895e-06 - accuracy: 1.0000
Epoch 9/10
50/50 [==============================] - 23s 463ms/step - loss: 3.6661e-06 - accuracy: 1.0000
Epoch 10/10
50/50 [==============================] - 21

In [59]:
#save model and weights
with open("model_7.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_7.h5")

In [60]:
#load model and weights
yaml_file7 = open('model_7.yaml', 'r')
loaded_model_yaml7 = yaml_file7.read()
yaml_file7.close()
model7 = tf.keras.models.model_from_yaml(loaded_model_yaml7)

model7.load_weights("model_7.h5")

In [61]:
# iterate over each test image
# make a prediction and add to results 
data = []
results7 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result7 = model7.predict(img)
    r7 = np.argmax(result7, axis=1)
    results7.append(r7)


result7 = list(itertools.chain.from_iterable(results7))
accuracy_rate7 = sum(correct_class == result7)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result7)
print("accuracy rate for model 7:", accuracy_rate7)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 2, 0, 1, 2, 1, 1]
accuracy rate for model 7: 0.375


## 3-8. steps_per_epoch: 50, epochs: 20

In [62]:
model_8 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=50,
            epochs=20)

Epoch 1/20
50/50 [==============================] - 21s 428ms/step - loss: 2.8892e-06 - accuracy: 1.0000
Epoch 2/20
50/50 [==============================] - 20s 402ms/step - loss: 3.9141e-06 - accuracy: 1.0000
Epoch 3/20
50/50 [==============================] - 20s 403ms/step - loss: 4.8645e-06 - accuracy: 1.0000
Epoch 4/20
50/50 [==============================] - 21s 422ms/step - loss: 0.0013 - accuracy: 0.9993
Epoch 5/20
50/50 [==============================] - 21s 414ms/step - loss: 0.0339 - accuracy: 0.9870
Epoch 6/20
50/50 [==============================] - 20s 405ms/step - loss: 5.6070e-04 - accuracy: 1.0000
Epoch 7/20
50/50 [==============================] - 20s 402ms/step - loss: 7.6062e-05 - accuracy: 1.0000
Epoch 8/20
50/50 [==============================] - 21s 420ms/step - loss: 4.9928e-05 - accuracy: 1.0000
Epoch 9/20
50/50 [==============================] - 25s 502ms/step - loss: 4.2128e-05 - accuracy: 1.0000
Epoch 10/20
50/50 [==============================] - 22s 443ms/

In [63]:
#save model and weights
with open("model_8.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_8.h5")

In [64]:
#load model and weights
yaml_file8 = open('model_8.yaml', 'r')
loaded_model_yaml8 = yaml_file8.read()
yaml_file8.close()
model8 = tf.keras.models.model_from_yaml(loaded_model_yaml8)

model8.load_weights("model_8.h5")

In [65]:
# iterate over each test image
# make a prediction and add to results 
data = []
results8 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result8 = model8.predict(img)
    r8 = np.argmax(result8, axis=1)
    results8.append(r8)


result8 = list(itertools.chain.from_iterable(results8))
accuracy_rate8 = sum(correct_class == result8)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result8)
print("accuracy rate for model 8:", accuracy_rate8)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 0, 0, 1, 2, 1, 1]
accuracy rate for model 8: 0.375


## 3-9.steps_per_epoch: 50, epochs: 30

In [66]:
model_9 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=50,
            epochs=30)

Epoch 1/30
50/50 [==============================] - 21s 417ms/step - loss: 3.8974e-06 - accuracy: 1.0000
Epoch 2/30
50/50 [==============================] - 21s 411ms/step - loss: 7.6562e-06 - accuracy: 1.0000
Epoch 3/30
50/50 [==============================] - 20s 407ms/step - loss: 2.8668e-06 - accuracy: 1.0000
Epoch 4/30
50/50 [==============================] - 20s 407ms/step - loss: 5.9319e-06 - accuracy: 1.0000
Epoch 5/30
50/50 [==============================] - 21s 417ms/step - loss: 1.2124e-05 - accuracy: 1.0000
Epoch 6/30
50/50 [==============================] - 21s 430ms/step - loss: 3.4099e-06 - accuracy: 1.0000
Epoch 7/30
50/50 [==============================] - 20s 404ms/step - loss: 1.1730e-06 - accuracy: 1.0000
Epoch 8/30
50/50 [==============================] - 20s 408ms/step - loss: 9.4762e-06 - accuracy: 1.0000
Epoch 9/30
50/50 [==============================] - 20s 404ms/step - loss: 2.5445e-06 - accuracy: 1.0000
Epoch 10/30
50/50 [==============================] - 20

In [67]:
#save model and weights
with open("model_9.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_9.h5")

In [68]:
#load model and weights
yaml_file9 = open('model_9.yaml', 'r')
loaded_model_yaml9 = yaml_file9.read()
yaml_file9.close()
model9 = tf.keras.models.model_from_yaml(loaded_model_yaml9)

model9.load_weights("model_9.h5")

In [69]:
# iterate over each test image
# make a prediction and add to results 
data = []
results9 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result9 = model9.predict(img)
    r9 = np.argmax(result9, axis=1)
    results9.append(r9)


result9 = list(itertools.chain.from_iterable(results9))
accuracy_rate9 = sum(correct_class == result9)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result9)
print("accuracy rate for model 9:", accuracy_rate9)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 0, 0, 1, 2, 1, 1]
accuracy rate for model 9: 0.375


## 3-10. steps_per_epoch: 50, epochs: 100

In [70]:
model_10 = classifier.fit_generator(
            train_generator,
            steps_per_epoch=50,
            epochs=100)

Epoch 1/100
50/50 [==============================] - 21s 415ms/step - loss: 9.0275e-07 - accuracy: 1.0000
Epoch 2/100
50/50 [==============================] - 20s 406ms/step - loss: 6.0166e-07 - accuracy: 1.0000
Epoch 3/100
50/50 [==============================] - 21s 419ms/step - loss: 8.7161e-07 - accuracy: 1.0000
Epoch 4/100
50/50 [==============================] - 21s 426ms/step - loss: 8.8025e-07 - accuracy: 1.0000
Epoch 5/100
50/50 [==============================] - 20s 403ms/step - loss: 5.5370e-07 - accuracy: 1.0000
Epoch 6/100
50/50 [==============================] - 21s 412ms/step - loss: 1.5155e-06 - accuracy: 1.0000
Epoch 7/100
50/50 [==============================] - 20s 408ms/step - loss: 5.3538e-07 - accuracy: 1.0000
Epoch 8/100
50/50 [==============================] - 20s 404ms/step - loss: 8.1390e-07 - accuracy: 1.0000
Epoch 9/100
50/50 [==============================] - 20s 401ms/step - loss: 5.5718e-07 - accuracy: 1.0000
Epoch 10/100
50/50 [==========================

In [71]:
#save model and weights
with open("model_10.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

classifier.save_weights("model_10.h5")

In [72]:
#load model and weights
yaml_file10 = open('model_10.yaml', 'r')
loaded_model_yaml10 = yaml_file10.read()
yaml_file10.close()
model10 = tf.keras.models.model_from_yaml(loaded_model_yaml10)

model10.load_weights("model_10.h5")

In [73]:
# iterate over each test image
# make a prediction and add to results 
data = []
results10 = []
for f1 in files:
    img = image.load_img(f1, target_size = (64, 64))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    data.append(img)
    result10 = model10.predict(img)
    r10 = np.argmax(result10, axis=1)
    results10.append(r10)


result10 = list(itertools.chain.from_iterable(results10))
accuracy_rate10 = sum(correct_class == result10)/len(correct_class)
print("correct classification", correct_class)
print("classification result", result10)
print("accuracy rate for model 10:", accuracy_rate10)

correct classification [3 0 1 0 2 1 3 2]
classification result [3, 0, 0, 0, 1, 2, 1, 1]
accuracy rate for model 10: 0.375


In [75]:
#f) 
import pandas as pd
data = {'Steps per Epoch':[10,10,10,30,30,30,50,50,50,50], 
        'Epochs':[10,20,30,10,20,30,10,20,30,100], 
        'Accuracy':[accuracy_rate1, accuracy_rate2, accuracy_rate3, accuracy_rate4, 
                    accuracy_rate5, accuracy_rate6, accuracy_rate7, accuracy_rate8, 
                    accuracy_rate9, accuracy_rate10]} 
pd.DataFrame(data)

,Steps per Epoch,Epochs,Accuracy
0,10,10,0.500
1,10,20,0.500
2,10,30,0.500
3,30,10,0.500
4,30,20,0.500
5,30,30,0.375
6,50,10,0.375
7,50,20,0.375
8,50,30,0.375
9,50,100,0.375


# Conceptual Questions: 

4. Discuss the effect of the following on accuracy and loss (train & test):

a) Increasing the steps_per_epoch

Answer: With increase in steps per epoch, the accuracy for train slightly increase (from close to 1 in previous models to all 1's in model 10). However, the testing accuracy is decreasing (from 0.5 in the earlier models to 0.375 in later models). Loss decreases as steps per epoch increases (from around e-02 in the earlier models to e-08 in the later models). 

b) Increasing the number of epochs

Answer: The increase in number of epochs does not have a significant effect on training and testing accuracy as compared to the increase in steps per epoch. For training, the same epochs can have various of training accuracy, and there is no sign of how number of epochs had effect on training accuracy. Similary, for testing, epochs = 10, 20, 30 can have testing accuracy of 0.5 as well as 0.375). In addition, number of epochs does not have significant effect on the loss (loss decreases as steps per epoch increase, but it does not have any significant pattern with number of epochs).

5. Name two uses of zero padding in CNN.

Answer:

Zero-padding is the process of symmetrically adding zeroes around the borders of the input matrix. It allows adjustment to the size of our input matrix the size. There are many uses of zero padding in CNN; the followings are just a few examples: 

a) Zero padding can be used when you want your output matrix to be the same size as your input matrix. For instance, if you have a input matrix of 4 * 4, and filter of 3 * 3, and output matrix should have a size of 2 * 2. If you use zero padding to your input matrix, the input matrix would have a size of 6 * 6, and with the same filter, your output matrix is also 4 * 4. 

b) Zero padding also help avoid loosing information at the boundaries. Without zero padding, the pixels at the borders are not trained as often as the pixels at the center, and hence, the border information might not be captured/ given weight as it should be.

c) Zero padding can also be used when we want to train a deeper network. With zero padding, the iterations at the borders are only training on some of the pixels (and some zeros), which help the network train faster and enable a deeper network. 


6. What is the use of a 1 x 1 kernel in CNN?

Answer: One of the use of a 1 x 1 kernel in CNN is for dimension reduction. For instance, if our input volumn is 63 * 64 * 192, and we apply a 1 * 1 kernel (in this case, it should be 1 * 1 * 192), then the output volumne should be 64 * 64 * 1. In this way, we reduce the depth while keeping the height and width of the feature map. This is reduce the computational needs, be more efficience and build deeper network. 


7. What are the advantages of a CNN over a fully connected DNN for this image classification problem?

Answer:

The main advantage of CNN compared to a fully connected DNN is that it automatically detects the important features without any human supervision. For example, given many pictures of cats and dogs it learns distinctive features for each class by itself.

Another main feature of CNNs is weight sharing, which help CNN to be more computationally efficent by computing less parameters. For instance, a one layered CNN with 5 filters of size 5x5, we have total of 5x5x5 +10 (bias) = 135 parameters to compute, which is way less when we consider if we have a fully connected DNN of a similar size. 